In [ ]:
import numpy as np
import cv2

def padder(matrix, x, y):
    height, width = matrix.shape
    out = np.zeros((height + x, width + y), dtype=np.uint8)
    for i in range(height + x):
        for j in range(width + y):
            if i < x or i >= height - x or j < y or j >= width - y:
                out[i, j] = 0
            else:
                out[i, j] = matrix[i - x, j - y]
    return out

def smoothing_filter(original, value):
    rows, columns = original.shape
    x, y = value, value
    filter_1 = np.ones((x, y)) / (x * y)
    padded_image = padder(original, x, y)
    out = np.ones_like(padded_image, dtype=np.uint8) * 255

    for i in range(rows):
        for j in range(columns):
            subimg = padded_image[i:i + x, j:j + y]
            product = np.multiply(subimg, filter_1)
            out[i + (x // 2), j + (y // 2)] = np.sum(product)
    return out

def normalize_image(image):
    return cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def sobel_filters(image):
    rows, columns = image.shape

    sobel_x = np.array([[-2, -1, 0], 
                        [-1,  0, 1], 
                        [ 0,  1, 2]], dtype=np.float32)

    sobel_y = np.array([[-1,  0,  1], 
                        [-2,  0,  2], 
                        [-1,  0,  1]], dtype=np.float32)

    padded_image = padder(image, 1, 1)  

    sobel_x_img = np.zeros_like(image, dtype=np.float32)
    sobel_y_img = np.zeros_like(image, dtype=np.float32)

    for i in range(rows - 2):
        for j in range(columns - 2):
            subimg = padded_image[i:i + 3, j:j + 3]
            sobel_x_img[i + 1, j + 1] = np.sum(subimg * sobel_x)
            sobel_y_img[i + 1, j + 1] = np.sum(subimg * sobel_y)

    magnitude = np.sqrt(sobel_x_img**2 + sobel_y_img**2)
    phase = np.arctan2(sobel_y_img, sobel_x_img) * 180 / np.pi  

    return magnitude, phase

def quantize_angle(angle):
    quantized = np.zeros_like(angle)
    angle = angle % 180  

    quantized[(angle >= 0) & (angle < 22.5)] = 0
    quantized[(angle >= 22.5) & (angle < 67.5)] = 45
    quantized[(angle >= 67.5) & (angle < 112.5)] = 90
    quantized[(angle >= 112.5) & (angle < 157.5)] = 135
    quantized[(angle >= 157.5)] = 0

    return quantized

def non_maximum_suppression(magnitude, angle):
    h, w = magnitude.shape
    output = np.zeros((h, w), dtype=np.float32)
    angle = quantize_angle(angle)

    for i in range(1, h - 1):
        for j in range(1, w - 1):
            q, r = 255, 255

            if angle[i, j] == 0:
                q = magnitude[i, j + 1]
                r = magnitude[i, j - 1]
            elif angle[i, j] == 90:
                q = magnitude[i + 1, j]
                r = magnitude[i - 1, j]
            elif angle[i, j] == 45:
                q = magnitude[i - 1, j + 1]
                r = magnitude[i + 1, j - 1]
            elif angle[i, j] == 135:
                q = magnitude[i - 1, j - 1]
                r = magnitude[i + 1, j + 1]

            if magnitude[i, j] >= q and magnitude[i, j] >= r:
                output[i, j] = magnitude[i, j]
            else:
                output[i, j] = 0

    return output

def double_threshold(image, low_threshold_ratio=0.01, high_threshold_ratio=0.05):
    max_value = np.max(image)
    high_threshold = max_value * high_threshold_ratio
    low_threshold = max_value * low_threshold_ratio

    strong_edges = (image >= high_threshold).astype(np.uint8)
    weak_edges = ((image >= low_threshold) & (image < high_threshold)).astype(np.uint8)

    return strong_edges, weak_edges

def edge_tracking_by_hysteresis(strong_edges, weak_edges):
    edges = np.copy(strong_edges)
    h, w = edges.shape

    for i in range(1, h - 1):
        for j in range(1, w - 1):
            if weak_edges[i, j] == 1:
                if (strong_edges[i-1:i+2, j-1:j+2] == 1).any():
                    edges[i, j] = 1

    return edges

def canny_edge_detection(image_path):
    
    image = cv2.imread(image_path, 0)
    blurred = smoothing_filter(image, 5) 
    magnitude, angle = sobel_filters(blurred)  
    suppressed = non_maximum_suppression(magnitude, angle)
    strong_edges, weak_edges = double_threshold(suppressed)
    edges = edge_tracking_by_hysteresis(strong_edges, weak_edges)

    return edges

image_path = "Fig03.tif" 
edges = canny_edge_detection(image_path)

cv2.imshow("Canny Edge Detection", edges * 255) 
cv2.waitKey()

-1